In [15]:
import pandas as pd
import data_loader as dl
import re
import math
import ast


In [2]:
#A good algrm that looks at shape will look for isotopes. WHat % of y peaks come with a +1 isotope?

In [3]:
#Read in data
bulk_rep1 = dl.load_joined_psm_mzml('bulk_rep1')
bulk_rep2 = dl.load_joined_psm_mzml('bulk_rep2')
bulk_rep3 = dl.load_joined_psm_mzml('bulk_rep3')

rep1_2ng = dl.load_joined_psm_mzml("2ng_rep1")
rep2_2ng = dl.load_joined_psm_mzml("2ng_rep2")
rep3_2ng = dl.load_joined_psm_mzml("2ng_rep3")
rep4_2ng = dl.load_joined_psm_mzml("2ng_rep4")
rep5_2ng = dl.load_joined_psm_mzml("2ng_rep5")
rep6_2ng = dl.load_joined_psm_mzml("2ng_rep6")

rep1_02ng = dl.load_joined_psm_mzml("0.2ng_rep1")
rep2_02ng = dl.load_joined_psm_mzml("0.2ng_rep2")
rep3_02ng = dl.load_joined_psm_mzml("0.2ng_rep3")
rep4_02ng = dl.load_joined_psm_mzml("0.2ng_rep4")
rep5_02ng = dl.load_joined_psm_mzml("0.2ng_rep5")
rep6_02ng = dl.load_joined_psm_mzml("0.2ng_rep6")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (12,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (12,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (12,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda

In [4]:
#filter based on retention time gradient
rep1_2ng = rep1_2ng[(rep1_2ng['minute'] < 95) & (rep1_2ng['minute'] >= 27)]
rep2_2ng = rep2_2ng[(rep2_2ng['minute'] < 95) & (rep2_2ng['minute'] >=27)]
rep3_2ng = rep3_2ng[(rep3_2ng['minute'] < 93) & (rep3_2ng['minute'] >=32)]
rep4_2ng = rep4_2ng[(rep4_2ng['minute'] < 91) & (rep4_2ng['minute'] >=32)]
rep5_2ng = rep5_2ng[(rep5_2ng['minute'] < 99) & (rep5_2ng['minute'] >=31)]
rep6_2ng = rep6_2ng[(rep6_2ng['minute'] < 98) & (rep6_2ng['minute'] >=31)]

rep1_02ng = rep1_02ng[(rep1_02ng['minute'] < 94) & (rep1_02ng['minute'] >=28)]
rep2_02ng = rep2_02ng[(rep2_02ng['minute'] < 95) & (rep2_02ng['minute'] >=27)]
rep3_02ng = rep3_02ng[(rep3_02ng['minute'] < 98) & (rep3_02ng['minute'] >=32)]
rep4_02ng = rep4_02ng[(rep4_02ng['minute'] < 98) & (rep4_02ng['minute'] >=32)]
rep5_02ng = rep5_02ng[(rep5_02ng['minute'] < 109) & (rep5_02ng['minute'] >=35)]
rep6_02ng = rep6_02ng[(rep6_02ng['minute'] < 109) & (rep6_02ng['minute'] >=35)]

In [5]:
rep1_2ng["minute_zeroed"] = rep1_2ng['minute'] - 27
rep2_2ng["minute_zeroed"] = rep2_2ng['minute'] - 27
rep3_2ng["minute_zeroed"] = rep3_2ng['minute'] - 32
rep4_2ng["minute_zeroed"] = rep4_2ng['minute'] - 32
rep5_2ng["minute_zeroed"] = rep5_2ng['minute'] - 31
rep6_2ng["minute_zeroed"] = rep6_2ng['minute'] - 31

rep1_02ng["minute_zeroed"] = rep1_02ng['minute'] - 28
rep2_02ng["minute_zeroed"] = rep2_02ng['minute'] - 27
rep3_02ng["minute_zeroed"] = rep3_02ng['minute'] - 32
rep4_02ng["minute_zeroed"] = rep4_02ng['minute'] - 32
rep5_02ng["minute_zeroed"] = rep5_02ng['minute'] - 35
rep6_02ng["minute_zeroed"] = rep6_02ng['minute'] - 35

In [6]:
#get a list of m/z for the y peaks
def get_y_mz(row):
    line = row['Matched Ion Mass-To-Charge Ratios']
    line = line.split(';')  
    if len(line) == 2:
        if 'y' in line[1]:
            y_peaks = line[1]
        else: #if there are no y peaks:
            return 'NAN'
            
    elif len(line) == 1:
        if 'y' in line[0]:
            y_peaks = line[0]
        else: #if there are no y peaks:
            return 'NAN'
    
    else: #if there are no y peaks:
        return 'NAN'
    
    
    y_mz = []
    el = y_peaks.strip('][').split(",")
    for sub_el in el:
        sub_el = sub_el.split(":")[1]
        y_mz.append(sub_el)

    return y_mz


In [19]:
def count_plus1_isotopes(row):
    plusone_count = 0
    allmz = row['mz_array']
    allmz = ast.literal_eval(allmz)
    if row['y_mz'] == 'NAN': return 'NAN'
    for ypeak in row['y_mz']:
            ypeak = float(ypeak)
            plusone = ypeak + 1.0034
            plusone_tol_max = plusone+.01
            plusone_tol_min = plusone-.01
            for test_mz in allmz:  
                if plusone_tol_min <= test_mz <= plusone_tol_max:
                    plusone_count +=1
    return plusone_count

In [20]:
def get_plus_one_wrapper(df):
    mz = df[['Matched Ion Mass-To-Charge Ratios','mz_array', 'scan','probability']]
    mz = mz.set_index("scan")
    mz['y_mz'] = mz.apply(get_y_mz, axis=1)
    mz['plusone_count'] = mz.apply(count_plus1_isotopes, axis=1)
    mz['count_all_y'] = mz.apply(lambda row: len(row['y_mz']), axis=1)
    mz = mz[mz['plusone_count'] != 'NAN'] #filter out rows that didn't have any y peaks
    mz['percent_with_plusone'] = mz['plusone_count']/mz['count_all_y'] #percent of peak in spectra that have a plus one
    
    #seprate good and medicore spectra
    df_good = mz[mz['probability'] <= .01][['percent_with_plusone']]
    df_med = mz[(mz['probability'] > .01) & (mz['probability'] <= .2)][['percent_with_plusone']]

    return [df_good, df_med]
    

In [21]:
rep1_02ng_plusone_good, rep1_02ng_plusone_med = get_plus_one_wrapper(rep1_02ng)
rep2_02ng_plusone_good, rep2_02ng_plusone_med = get_plus_one_wrapper(rep2_02ng)
rep3_02ng_plusone_good, rep3_02ng_plusone_med = get_plus_one_wrapper(rep3_02ng)
rep4_02ng_plusone_good, rep4_02ng_plusone_med = get_plus_one_wrapper(rep4_02ng)
rep5_02ng_plusone_good, rep5_02ng_plusone_med = get_plus_one_wrapper(rep5_02ng)
rep6_02ng_plusone_good, rep6_02ng_plusone_med = get_plus_one_wrapper(rep6_02ng)

rep1_2ng_plusone_good, rep1_2ng_plusone_med = get_plus_one_wrapper(rep1_2ng)
rep2_2ng_plusone_good, rep2_2ng_plusone_med = get_plus_one_wrapper(rep2_2ng)
rep3_2ng_plusone_good, rep3_2ng_plusone_med = get_plus_one_wrapper(rep3_2ng)
rep4_2ng_plusone_good, rep4_2ng_plusone_med = get_plus_one_wrapper(rep4_2ng)
rep5_2ng_plusone_good, rep5_2ng_plusone_med = get_plus_one_wrapper(rep5_2ng)
rep6_2ng_plusone_good, rep6_2ng_plusone_med = get_plus_one_wrapper(rep6_2ng)

bulkrep1_plusone_good, bulkrep1_plusone_med = get_plus_one_wrapper(bulk_rep1)
bulkrep2_plusone_good, bulkrep2_plusone_med = get_plus_one_wrapper(bulk_rep2)
bulkrep3_plusone_good, bulkrep3_plusone_med = get_plus_one_wrapper(bulk_rep3)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [22]:
all02ng_good = pd.concat([rep1_02ng_plusone_good,rep2_02ng_plusone_good,rep3_02ng_plusone_good,
                        rep4_02ng_plusone_good,rep5_02ng_plusone_good,rep6_02ng_plusone_good])
all02ng_med = pd.concat([rep1_02ng_plusone_med,rep2_02ng_plusone_med,rep3_02ng_plusone_med,
                        rep4_02ng_plusone_med,rep5_02ng_plusone_med,rep6_02ng_plusone_med])

all2ng_good = pd.concat([rep1_2ng_plusone_good,rep2_2ng_plusone_good,rep3_2ng_plusone_good,
                        rep4_2ng_plusone_good,rep5_2ng_plusone_good,rep6_2ng_plusone_good])
all2ng_med = pd.concat([rep1_2ng_plusone_med,rep2_2ng_plusone_med,rep3_2ng_plusone_med,
                        rep4_2ng_plusone_med,rep5_2ng_plusone_med,rep6_2ng_plusone_med])

all_bulk_good = pd.concat([bulkrep1_plusone_good,bulkrep2_plusone_good,bulkrep3_plusone_good])
all_bulk_med = pd.concat([bulkrep1_plusone_med,bulkrep2_plusone_med,bulkrep3_plusone_med])


In [24]:
print('Bulk percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak')
len(all_bulk_good[all_bulk_good['percent_with_plusone'] >= .5])/len(all_bulk_good)

Bulk percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak


0.6693192382009238

In [25]:
print('2ng good percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak')
len(all2ng_good[all2ng_good['percent_with_plusone'] >= .5])/len(all2ng_good)

2ng good percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak


0.5127619838140693

In [26]:
print('2ng mediocre percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak')
len(all2ng_med[all2ng_med['percent_with_plusone'] >= .5])/len(all2ng_med)

2ng mediocre percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak


0.32067351659583626

In [27]:
print('2ng quality 20% or fewer of the y ions')
len(all2ng_good[all2ng_good['percent_with_plusone'] <= .2])/len(all2ng_good)

2ng quality 20% or fewer of the y ions


0.15822784810126583

In [28]:
print('2ng mediocre 20% or fewer of the y ions')
len(all2ng_med[all2ng_med['percent_with_plusone'] <= .2])/len(all2ng_med)

2ng mediocre 20% or fewer of the y ions


0.41465382753241165

# not in paper

In [29]:
print('0.2ng good percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak')
len(all02ng_good[all02ng_good['percent_with_plusone'] >= .5])/len(all02ng_good)

0.2ng good percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak


0.39798314268512947

In [30]:
print('0.2ng mediocre percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak')
len(all02ng_med[all02ng_med['percent_with_plusone'] >= .5])/len(all02ng_med)

0.2ng mediocre percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak


0.25934579439252337

In [31]:
print('0.2ng quality 20% or fewer of the y ions')
len(all02ng_good[all02ng_good['percent_with_plusone'] <= .2])/len(all02ng_good)

0.2ng quality 20% or fewer of the y ions


0.21270319084888623

In [32]:
print('0.2ng mediocre 20% or fewer of the y ions')
len(all02ng_med[all02ng_med['percent_with_plusone'] <= .2])/len(all02ng_med)

0.2ng mediocre 20% or fewer of the y ions


0.443500424808836

In [33]:
print('Bulk mediocre percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak')
len(all_bulk_med[all_bulk_med['percent_with_plusone'] >= .5])/len(all_bulk_med)

Bulk mediocre percent PSMs that have at least 50% of all y ions observed with a +1 isotopic peak


0.42766967499608743

In [34]:
print('Bulk quality 20% or fewer of the y ions')
len(all_bulk_good[all_bulk_good['percent_with_plusone'] <= .2])/len(all_bulk_good)

Bulk quality 20% or fewer of the y ions


0.10860941979256404

In [35]:
print('Bulk mediocre 20% or fewer of the y ions')
len(all_bulk_med[all_bulk_med['percent_with_plusone'] <= .2])/len(all_bulk_med)

Bulk mediocre 20% or fewer of the y ions


0.36326012485436554